In [29]:
# ASL Basic Phrases → TensorFlow Deep Learning Version

import os
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

# === 1️⃣ Dataset Loading & Combining ===
DATA_DIR = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\ASLBasicPhrasesSignLanguages\Time"
all_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))

print(f"📁 Found {len(all_files)} dataset files")

df_list = []
for file in all_files:
    try:
        df = pd.read_csv(file)
        if df.empty:
            print(f"⚠️ Skipped empty file: {os.path.basename(file)}")
            continue
        df_list.append(df)
        print(f"✅ Loaded {os.path.basename(file)} ({df.shape[0]} samples)")
    except Exception as e:
        print(f"❌ Error reading {os.path.basename(file)}: {e}")

# Combine all
if not df_list:
    raise ValueError("🚫 No valid datasets found to merge.")

final_df = pd.concat(df_list, ignore_index=True)
combined_path = os.path.join(DATA_DIR, "combined_basic_phrases_dataset.csv")
final_df.to_csv(combined_path, index=False)

print("\n✅ Combined dataset created successfully!")
print(f"📄 Saved to: {combined_path}")
print("🧮 Total samples:", final_df.shape[0])
print("🏷️ Labels:", final_df['label'].unique())

# === 2️⃣ Preprocessing ===
print("\n🔧 Cleaning and preparing data...")

# Drop NaNs and ensure numeric
final_df = final_df.dropna()
X = final_df.drop('label', axis=1)
X = X.apply(pd.to_numeric, errors='coerce').fillna(0).values

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(final_df['label'])

# Save label encoder for later decoding
np.save(os.path.join(DATA_DIR, "Time_classes.npy"), label_encoder.classes_)
print("💾 Saved label classes for later decoding")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# === 3️⃣ Build TensorFlow Model ===
print("\n🧠 Building TensorFlow Model...")

model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(y)), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# === 4️⃣ Train Model ===
print("\n🚀 Training model...")
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=32,
    verbose=1
)

# === 5️⃣ Evaluate ===
print("\n📊 Evaluating model...")
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {test_acc:.4f}")
print(f"📉 Test Loss: {test_loss:.4f}")

# === 6️⃣ Save Models (Keras, SavedModel, TFLite) ===
TFMODELS_DIR = os.path.join(DATA_DIR, "TimeModels")
os.makedirs(TFMODELS_DIR, exist_ok=True)

KERAS_PATH = os.path.join(TFMODELS_DIR, "ASL_Time_Model.keras")
SAVEDMODEL_PATH = os.path.join(TFMODELS_DIR, "ASL_Time_SavedModel")
TFLITE_PATH = os.path.join(TFMODELS_DIR, "ASL_Time_Model.tflite")

# Save .keras
model.save(KERAS_PATH)
print(f"💾 Saved Keras model → {KERAS_PATH}")

# Save as TensorFlow SavedModel
model.export(SAVEDMODEL_PATH)
print(f"💾 Saved TensorFlow SavedModel → {SAVEDMODEL_PATH}")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open(TFLITE_PATH, "wb") as f:
    f.write(tflite_model)

print(f"💾 Saved TFLite model → {TFLITE_PATH}")

print("\n✅ All models exported successfully!")


📁 Found 7 dataset files
✅ Loaded afternoon.csv (296 samples)
❌ Error reading combined_basic_phrases_dataset.csv: No columns to parse from file
✅ Loaded morning.csv (303 samples)
✅ Loaded night.csv (302 samples)
✅ Loaded today.csv (227 samples)
✅ Loaded tommorow.csv (295 samples)
✅ Loaded yesterday.csv (286 samples)

✅ Combined dataset created successfully!
📄 Saved to: C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\ASLBasicPhrasesSignLanguages\Time\combined_basic_phrases_dataset.csv
🧮 Total samples: 1709
🏷️ Labels: ['afternoon' 'morning' 'night' 'today' 'tommorow' 'yesterday']

🔧 Cleaning and preparing data...
💾 Saved label classes for later decoding

🧠 Building TensorFlow Model...


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_48 (Dense)                │ (None, 128)            │        16,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,902 (97.27 KB)

 Trainable params: 24,902 (97.27 KB)

...


🚀 Training model...
Epoch 1/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.4648 - loss: 1.4193 - val_accuracy: 0.6971 - val_loss: 1.0487
Epoch 2/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6780 - loss: 0.9568 - val_accuracy: 0.7336 - val_loss: 0.8392
Epoch 3/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7466 - loss: 0.8156 - val_accuracy: 0.7737 - val_loss: 0.7457
Epoch 4/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7685 - loss: 0.7099 - val_accuracy: 0.7701 - val_loss: 0.6741
Epoch 5/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7914 - loss: 0.6472 - val_accuracy: 0.7920 - val_loss: 0.6260
Epoch 6/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7887 - loss: 0.5986 - val_accuracy: 0.7847 - val_loss: 0.5740
Epoch 7/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.7969 - loss: 0.5731 - val_accuracy: 0.7920 - val_loss: 0.5342
Epoch 8/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8042 - loss: 0.5319 - val_ac

INFO:tensorflow:Assets written to: C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\ASLBasicPhrasesSignLanguages\Time\TimeModels\ASL_Time_SavedModel\assets


Saved artifact at 'C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\ASLBasicPhrasesSignLanguages\Time\TimeModels\ASL_Time_SavedModel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 126), dtype=tf.float32, name='keras_tensor_170')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  1438288452624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1438288460304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1438288454736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1438267497488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1438267499408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1438267499024: TensorSpec(shape=(), dtype=tf.resource, name=None)
💾 Saved TensorFlow SavedModel → C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\ASLBasicPhrasesSignLanguages\Time\TimeModels\ASL_Time_SavedModel
INFO:tensorflow

INFO:tensorflow:Assets written to: C:\Users\JAMJAY~1\AppData\Local\Temp\tmpnsxfepfc\assets


Saved artifact at 'C:\Users\JAMJAY~1\AppData\Local\Temp\tmpnsxfepfc'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 126), dtype=tf.float32, name='keras_tensor_170')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  1438288452624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1438288460304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1438288454736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1438267497488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1438267499408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1438267499024: TensorSpec(shape=(), dtype=tf.resource, name=None)
💾 Saved TFLite model → C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\ASLBasicPhrasesSignLanguages\Time\TimeModels\ASL_Time_Model.tflite

✅ All models exported successfully!


In [38]:
# Real-Time ASL Phrase Prediction (Two-Hand Version) using TensorFlow

import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from collections import deque

# === Paths ===
MODEL_PATH = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\Tensorflow\TFModels\ObjectThingsModels\ASL_ObjectThings_Model.keras"
LABEL_PATH = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\ASLBasicPhrasesSignLanguages\ObjectThings\ObjectThings_classes.npy"

# === Load model and labels ===
print("📦 Loading TensorFlow model...")
model = tf.keras.models.load_model(MODEL_PATH)
label_classes = np.load(LABEL_PATH, allow_pickle=True)
print(f"✅ Loaded model with {len(label_classes)} output labels")

# === MediaPipe Setup ===
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,                 # ✅ Allow both hands
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

# === Helper: Extract both-hand keypoints (126 features = 2 × 21 × 3) ===
def extract_two_hand_keypoints(results):
    left_hand = np.zeros(21 * 3)
    right_hand = np.zeros(21 * 3)

    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            label = results.multi_handedness[hand_idx].classification[0].label
            coords = []
            for lm in hand_landmarks.landmark:
                coords.extend([lm.x, lm.y, lm.z])

            if label.lower() == 'left':
                left_hand = np.array(coords)
            else:
                right_hand = np.array(coords)

    # Always return fixed-length 126-dim vector
    return np.concatenate([left_hand, right_hand])

# === Prediction Smoothing ===
predictions_queue = deque(maxlen=10)

# === Start Webcam ===
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("🚫 Cannot access webcam.")
    exit()

print("🎥 Starting webcam... Press 'q' to quit.")
print("🖐 Show both hands clearly to the camera.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Frame capture failed, skipping...")
        continue

    # Flip and preprocess
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    # === Extract hand features (always 126 features) ===
    features = extract_two_hand_keypoints(results).reshape(1, -1)

    # === Predict if at least one hand is detected ===
    if np.any(features):
        probs = model.predict(features, verbose=0)
        pred_idx = np.argmax(probs)
        pred_label = label_classes[pred_idx]
        confidence = probs[0][pred_idx]

        predictions_queue.append(pred_label)
        stable_prediction = max(set(predictions_queue), key=predictions_queue.count)

        # Display text
        cv2.putText(frame,
                    f"{stable_prediction} ({confidence*100:.1f}%)",
                    (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0,
                    (0, 255, 0), 2)
    else:
        cv2.putText(frame, "No Hands Detected",
                    (20, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    1.0, (0, 0, 255), 2)

    # === Draw landmarks for both hands ===
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0),
                                       thickness=2, circle_radius=3),
                mp_drawing.DrawingSpec(color=(255, 0, 0),
                                       thickness=2)
            )

    # Show live frame
    cv2.imshow("ASL Phrase Recognition (TensorFlow Two-Hand)", frame)

    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# === Cleanup ===
cap.release()
cv2.destroyAllWindows()
hands.close()
print("🛑 Webcam closed.")


📦 Loading TensorFlow model...
✅ Loaded model with 7 output labels
🎥 Starting webcam... Press 'q' to quit.
🖐 Show both hands clearly to the camera.
🛑 Webcam closed.
